In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
# Checking the value count for the first column
df["koi_disposition"].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [6]:
# Filter out err columns
planets_filter_out_err = df.filter(regex = '^((?!err).)*$')
planets_filter_out_err.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


In [7]:
# Set the X and y 
X = planets_filter_out_err.drop(columns = ["koi_disposition"])
y = planets_filter_out_err["koi_disposition"]

print(X.shape, y.shape)

(6991, 20) (6991,)


# Create a Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify =y)

In [9]:
y.head()

0         CONFIRMED
1    FALSE POSITIVE
2    FALSE POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [11]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# # Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [12]:
y_train_categorical.shape

(5243, 3)

# Train the Model



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=3, activation='softmax'))

In [14]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 2,403
Trainable params: 2,403
Non-trainable params: 0
_________________________________________________________________


In [16]:
from keras.callbacks import EarlyStopping

callbacks =[EarlyStopping('val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks= callbacks,
    epochs=150,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/150


Using TensorFlow backend.


5243/5243 - 0s - loss: 0.6926 - accuracy: 0.6735
Epoch 2/150
5243/5243 - 0s - loss: 0.4290 - accuracy: 0.7860


C:\Users\edwin\anaconda3\envs\PythonAdv\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 3/150
5243/5243 - 0s - loss: 0.3914 - accuracy: 0.7950
Epoch 4/150
5243/5243 - 0s - loss: 0.3812 - accuracy: 0.7948
Epoch 5/150
5243/5243 - 0s - loss: 0.3767 - accuracy: 0.7994
Epoch 6/150
5243/5243 - 0s - loss: 0.3745 - accuracy: 0.8030
Epoch 7/150
5243/5243 - 0s - loss: 0.3725 - accuracy: 0.7967
Epoch 8/150
5243/5243 - 0s - loss: 0.3705 - accuracy: 0.8034
Epoch 9/150
5243/5243 - 0s - loss: 0.3683 - accuracy: 0.8087
Epoch 10/150
5243/5243 - 0s - loss: 0.3668 - accuracy: 0.8089
Epoch 11/150
5243/5243 - 0s - loss: 0.3665 - accuracy: 0.7995
Epoch 12/150
5243/5243 - 0s - loss: 0.3653 - accuracy: 0.8077
Epoch 13/150
5243/5243 - 0s - loss: 0.3635 - accuracy: 0.8097
Epoch 14/150
5243/5243 - 0s - loss: 0.3618 - accuracy: 0.8091
Epoch 15/150
5243/5243 - 0s - loss: 0.3618 - accuracy: 0.8156
Epoch 16/150
5243/5243 - 0s - loss: 0.3607 - accuracy: 0.8123
Epoch 17/150
5243/5243 - 0s - loss: 0.3611 - accuracy: 0.8104
Epoch 18/150
5243/5243 - 0s - loss: 0.3611 - accuracy: 0.8116
Epoch 19/150
52

Epoch 135/150
5243/5243 - 0s - loss: 0.2950 - accuracy: 0.8535
Epoch 136/150
5243/5243 - 0s - loss: 0.2951 - accuracy: 0.8570
Epoch 137/150
5243/5243 - 0s - loss: 0.2938 - accuracy: 0.8581
Epoch 138/150
5243/5243 - 0s - loss: 0.2928 - accuracy: 0.8594
Epoch 139/150
5243/5243 - 0s - loss: 0.2947 - accuracy: 0.8591
Epoch 140/150
5243/5243 - 0s - loss: 0.2927 - accuracy: 0.8583
Epoch 141/150
5243/5243 - 0s - loss: 0.2933 - accuracy: 0.8556
Epoch 142/150
5243/5243 - 0s - loss: 0.2927 - accuracy: 0.8583
Epoch 143/150
5243/5243 - 0s - loss: 0.2902 - accuracy: 0.8583
Epoch 144/150
5243/5243 - 0s - loss: 0.2908 - accuracy: 0.8613
Epoch 145/150
5243/5243 - 0s - loss: 0.2894 - accuracy: 0.8583
Epoch 146/150
5243/5243 - 0s - loss: 0.2888 - accuracy: 0.8619
Epoch 147/150
5243/5243 - 0s - loss: 0.2885 - accuracy: 0.8583
Epoch 148/150
5243/5243 - 0s - loss: 0.2868 - accuracy: 0.8644
Epoch 149/150
5243/5243 - 0s - loss: 0.2887 - accuracy: 0.8613
Epoch 150/150
5243/5243 - 0s - loss: 0.2870 - accuracy:

## Deep Learning

In [17]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=20))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))

In [18]:
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 12,503
Trainable params: 12,503
Non-trainable params: 0
_________________________________________________________________


In [19]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks= callbacks,
    epochs=150,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/150
5243/5243 - 0s - loss: 0.5141 - accuracy: 0.7309
Epoch 2/150
5243/5243 - 0s - loss: 0.3845 - accuracy: 0.7900


C:\Users\edwin\anaconda3\envs\PythonAdv\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 3/150
5243/5243 - 0s - loss: 0.3768 - accuracy: 0.7963
Epoch 4/150
5243/5243 - 0s - loss: 0.3723 - accuracy: 0.7995
Epoch 5/150
5243/5243 - 0s - loss: 0.3662 - accuracy: 0.8051
Epoch 6/150
5243/5243 - 0s - loss: 0.3643 - accuracy: 0.8047
Epoch 7/150
5243/5243 - 0s - loss: 0.3628 - accuracy: 0.8133
Epoch 8/150
5243/5243 - 0s - loss: 0.3595 - accuracy: 0.8085
Epoch 9/150
5243/5243 - 0s - loss: 0.3585 - accuracy: 0.8131
Epoch 10/150
5243/5243 - 0s - loss: 0.3551 - accuracy: 0.8154
Epoch 11/150
5243/5243 - 0s - loss: 0.3533 - accuracy: 0.8171
Epoch 12/150
5243/5243 - 0s - loss: 0.3520 - accuracy: 0.8167
Epoch 13/150
5243/5243 - 0s - loss: 0.3487 - accuracy: 0.8217
Epoch 14/150
5243/5243 - 0s - loss: 0.3506 - accuracy: 0.8173
Epoch 15/150
5243/5243 - 0s - loss: 0.3468 - accuracy: 0.8158
Epoch 16/150
5243/5243 - 0s - loss: 0.3541 - accuracy: 0.8110
Epoch 17/150
5243/5243 - 0s - loss: 0.3439 - accuracy: 0.8226
Epoch 18/150
5243/5243 - 0s - loss: 0.3448 - accuracy: 0.8222
Epoch 19/150
52

Epoch 135/150
5243/5243 - 0s - loss: 0.2111 - accuracy: 0.9027
Epoch 136/150
5243/5243 - 0s - loss: 0.2118 - accuracy: 0.9043
Epoch 137/150
5243/5243 - 0s - loss: 0.2099 - accuracy: 0.9044
Epoch 138/150
5243/5243 - 0s - loss: 0.2108 - accuracy: 0.9043
Epoch 139/150
5243/5243 - 0s - loss: 0.2138 - accuracy: 0.9016
Epoch 140/150
5243/5243 - 0s - loss: 0.2064 - accuracy: 0.9094
Epoch 141/150
5243/5243 - 0s - loss: 0.2107 - accuracy: 0.9041
Epoch 142/150
5243/5243 - 0s - loss: 0.2090 - accuracy: 0.9054
Epoch 143/150
5243/5243 - 0s - loss: 0.2029 - accuracy: 0.9100
Epoch 144/150
5243/5243 - 0s - loss: 0.2021 - accuracy: 0.9090
Epoch 145/150
5243/5243 - 0s - loss: 0.2050 - accuracy: 0.9058
Epoch 146/150
5243/5243 - 0s - loss: 0.2000 - accuracy: 0.9128
Epoch 147/150
5243/5243 - 0s - loss: 0.2045 - accuracy: 0.9105
Epoch 148/150
5243/5243 - 0s - loss: 0.2038 - accuracy: 0.9119
Epoch 149/150
5243/5243 - 0s - loss: 0.1986 - accuracy: 0.9128
Epoch 150/150
5243/5243 - 0s - loss: 0.2010 - accuracy:

## Compare the models below

In [20]:
 model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.3101 - accuracy: 0.8307
Normal Neural Network - Loss: 0.31009691667229417, Accuracy: 0.8306636214256287


In [21]:
 model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.2787 - accuracy: 0.8799
Deep Neural Network - Loss: 0.2786533906885361, Accuracy: 0.8798627257347107


## Saving Model

In [22]:
# Save the model
model.save("space_model_trained.h5")

In [23]:
# Save the model
deep_model.save("space_deep_model_trained.h5")

In [24]:
# Load the model
from tensorflow.keras.models import load_model
space_model = load_model("space_model_trained.h5")
space_deep_model = load_model("space_deep_model_trained.h5")

## Evaluating the loadedmodel

In [25]:
model_loss, model_accuracy = space_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.3101 - accuracy: 0.8307
Normal Neural Network - Loss: 0.31009691667229417, Accuracy: 0.8306636214256287


In [26]:
model_loss, model_accuracy = space_deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.2787 - accuracy: 0.8799
Normal Neural Network - Loss: 0.2786533906885361, Accuracy: 0.8798627257347107
